## **1. 라이브러리 설치**

In [ ]:
!pip install koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.5 MB/s eta 0:00:00


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from google.colab import files
import os
import zipfile
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import yaml
import shutil
from tqdm import tqdm
from glob import glob
import random
import numpy as np
import torch
from ultralytics import YOLO
from IPython.display import Image, display
import glob
import koreanize_matplotlib

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## **2. 시드 고정**

In [ ]:
random.seed(71)
np.random.seed(71)
torch.manual_seed(71)
torch.cuda.manual_seed(71)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## **3. 데이터 경로 설정**

In [ ]:
# 데이터 경로
data_path = '/content/drive/MyDrive/codeit_level1_project/data'

# 훈련, 검증, 테스트 데이터 경로 설정
train_dir = os.path.join(data_path, 'train_images')
test_dir = os.path.join(data_path, 'test_images')
annotation_dir = os.path.join(data_path, 'train_annotations')

## **4. 하이퍼파라미터 조정**
- 본 실험에서는 최종 훈련 모델은 YOLOv8x로 설정하되,
튜닝 시간 단축과 자원 절약을 위해 YOLOv8n 모델로 하이퍼파라미터 튜닝을 진행하였다.
- 튜닝에는 Ultralytics에서 제공하는 model.tune() 기능을 사용하였으며, 총 50회 반복 탐색(iterations=50)을 수행하였다.

| 분류             | 파라미터                                                                         | 이유                             |
| -------------- | ---------------------------------------------------------------------------- | ------------------------------ |
| 학습률 최적화     | `lr0`, `lrf`, `momentum`, `weight_decay`, `warmup_epochs`, `warmup_momentum` | 학습 속도 및 초기 안정성 개선                  |
| 손실 함수 가중치   | `box`, `cls`                                                                 | 바운딩 박스 회귀와 클래스 분류 간 손실 균형 조        |
| 데이터 증강       | `hsv_h`, `hsv_s`, `hsv_v`, `fliplr`, `mosaic`                                | 다양한 시각적 변형을 통해 모델 일반화 능력 향상            |


In [ ]:
model = YOLO("yolov8n.pt")

search_space = {
    "lr0": (1e-5, 1e-1),
    "lrf": (0.01, 1.0),
    "momentum": (0.6, 0.98),
    "weight_decay": (0.0, 0.001),
    "warmup_epochs": (0.0, 5.0),
    "warmup_momentum": (0.0, 0.95),
    "box": (0.02, 0.2),
    "cls": (0.2, 4.0),
    "hsv_h": (0.0, 0.1),
    "hsv_s": (0.0, 0.9),
    "hsv_v": (0.0, 0.9),
    "fliplr": (0.0, 1.0),
    "mosaic": (0.0, 1.0)
}

model.tune(
    data="/content/drive/MyDrive/codeit_level1_project/data/data.yaml",
    epochs=40,
    iterations=50,
    imgsz=640,
    optimizer="AdamW",
    space=search_space,
    save=True,
    val=True,
    plots=True,
    project="/content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/yolo_tuning_results",
    name="yolo8_hyp_0534"

)

100%|██████████| 6.25M/6.25M [00:00<00:00, 390MB/s]

Tuner: Initialized Tuner instance with 'tune_dir=/content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/yolo_tuning_results/yolo8_hyp_0534'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/50 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 0.2, 'cls': 0.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'fliplr': 0.5, 'mosaic': 1.0}


Saved /content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/yolo_tuning_results/yolo8_hyp_0534/tune_scatter_plots.png
Saved /content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/yolo_tuning_results/yolo8_hyp_0534/tune_fitness.png

Tuner: 1/50 iterations complete ✅ (589.97s)
Tuner: Results saved to /content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/yolo_tuning_results/yolo8_hyp_0534
Tuner: Best fitness=0.84648 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.72091, 'metrics/recall(B)': 0.97336, 'metrics/mAP50(B)': 0.86256, 'metrics/mAP50-95(B)': 0.8447, 'val/box_loss': 0.0065, 'val/cls_loss': 0.50605, 'val/dfl_loss': 0.78559, 'fitness': 0.84648}
Tuner: Best fitness model is /content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/yolo_tuning_results/train
Tuner: Best fitness hyperparameters are printed below.

Printing '/content/drive/MyDrive/codeit_level1_projec

## **5. YOLOv8x 모델 훈련 및 검증**

### 5.1 모델 구성 및 학습

- 앞서 튜닝한 하이퍼파라미터를 기반으로, YOLOv8x 모델을 활용하여 훈련을 진행하였다.
- 하이퍼파라미터는 model.tune()을 통해 YOLOv8n 모델로 탐색한 결과 중, 성능이 가장 우수했던 설정값을 적용하였다.
- 훈련은 총 100 epoch을 기준으로 하되, 5 epoch 동안 성능 개선이 없을 경우 자동 중단(early stopping) 되도록 설정하였다.

In [ ]:
model = YOLO("yolov8x.pt")

model.train(
    data='/content/drive/MyDrive/codeit_level1_project/data/data.yaml',
    imgsz=640,
    batch=16,
    workers=4,                 # 데이터 로딩 멀티스레딩
    patience=5,                # early stopping: 개선 없으면 조기 종료
    optimizer='AdamW',          # 옵티마이저
    seed=71,                   # 시드 고정으로 재현성 확보
    save=True,                 # 모델 체크포인트 저장 (기본 True)
    save_period=5,             # 매 5 에폭마다 저장
    project="/content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/pill_yolo_exp",   # 프로젝트 폴더명
    name="yolov8x_hyp0534_ep100",       # 실험 이름
    verbose=True,               # 학습 로그 출력
    lr0=0.0092,
    lrf=0.01,
    momentum=0.76932,
    weight_decay=0.0005,
    warmup_epochs=2.67276,
    warmup_momentum=0.74631,
    box=0.14182,
    cls=0.38274,
    hsv_h=0.01582,
    hsv_s=0.73822,
    hsv_v=0.28024,
    fliplr=0.55256,
    mosaic=1.0
)

Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=0.14182, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.38274, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/codeit_level1_project/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.55256, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.01582, hsv_s=0.73822, hsv_v=0.28024, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0092, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8x.pt, momentum=0.76932, mosaic=1.0, multi_scale=False, name=yolov8x_hyp0534_ep100, nbs=64, nms=False, opset=None, optimize=False, optimizer=Ad

100%|██████████| 22.2M/22.2M [00:00<00:00, 41.6MB/s]

Overriding model.yaml nc=80 with nc=73

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              


  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics.nn.modules.conv.Conv             [640, 640, 3, 2]              
  8                  -1  3   6969600  ultralytics.nn.modules.block.C2f             [640, 640, 3, True]           
  9                  -1  1   1025920  ultralytics.nn.modules.block.SPPF            [640, 640, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  3   7379200  ultralytics.nn.modules.block.C2f             [1280, 640, 3]                
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

100%|██████████| 5.35M/5.35M [00:00<00:00, 314MB/s]


...

                   all        295        890      0.698      0.934      0.834      0.807
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 48, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



53 epochs completed in 0.414 hours.
Optimizer stripped from /content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/pill_yolo_exp/yolov8x_hyp0534_ep100/weights/last.pt, 136.8MB
Optimizer stripped from /content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/pill_yolo_exp/yolov8x_hyp0534_ep100/weights/best.pt, 136.8MB

Validating /content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/pill_yolo_exp/yolov8x_hyp0534_ep100/weights/best.pt...
Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 112 layers, 68,193,867 parameters, 0 gradients, 257.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:02<00:00,  3.35it/s]


                   all        295        890      0.711      0.953      0.852      0.825
            보령부스파정 5mg         24         24      0.669          1      0.919      0.909
           뮤테란캡슐 100mg          1          1      0.811          1      0.995      0.995
           일양하이트린정 2mg          2          2       0.45          1      0.497      0.497
    기넥신에프정(은행엽엑스)(수출용)         90         90      0.728          1      0.777      0.771
     무코스타정(레바미피드)(비매품)          5          5      0.628          1      0.862      0.829
                  알드린정         10         10      0.487          1      0.647      0.647
          뉴로메드정(옥시라세탐)         10         10      0.794          1      0.889      0.841
            타이레놀정500mg          4          4          1      0.447        0.7      0.644
          에어탈정(아세클로페낙)         11         11      0.892      0.818      0.931      0.885
         삼남건조수산화알루미늄겔정         35         35      0.796          1       0.87      0.859
타이레놀이알서방정(아세트아미노펜)(수출

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7bf50db1a650>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0

### 5.2 검증 데이터 성능 평가
- 훈련 완료 후, 검증 데이터셋(295장, 890개 객체)을 대상으로 모델 성능을 평가하였다.

- 평가에는 튜닝된 모델의 best weight(best.pt)를 로드하여 model.val() 함수를 사용하였다.

- 검증 과정에서 일부 라벨이 비정상 범위를 가지는 이미지 2장은 자동으로 무시되었다.

| 지표                | 값     | 설명                         |
| ----------------- | ----- | -------------------------- |
| **Precision**     | 0.710 | 예측한 것 중 정답인 비율             |
| **Recall**        | 0.953 | 실제 객체 중 예측에 성공한 비율         |
| **mAP\@0.5**      | 0.852 | IoU ≥ 0.5일 때의 평균 정밀도       |
| **mAP\@0.5:0.95** | 0.827 | IoU 0.5\~0.95 범위에서의 평균 정밀도 |


In [ ]:
model = YOLO("/content/drive/MyDrive/codeit_level1_project/YOLOv8_epoch100_hyperparameters/pill_yolo_exp/yolov8x_hyp0534_ep100/weights/best.pt")
metrics = model.val()
print(metrics.box)

Ultralytics 8.3.146 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 112 layers, 68,193,867 parameters, 0 gradients, 257.8 GFLOPs


100%|██████████| 22.2M/22.2M [00:00<00:00, 136MB/s] 


val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 85.8±181.1 MB/s, size: 1690.6 KB)


val: Scanning /content/drive/MyDrive/codeit_level1_project/data/labels/val.cache... 297 images, 0 backgrounds, 2 corrupt: 100%|██████████| 297/297 [00:00<?, ?it/s]

val: /content/drive/MyDrive/codeit_level1_project/data/images/val/K-003351-016262-018357_0_2_0_2_75_000_200.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     6.8878]
val: /content/drive/MyDrive/codeit_level1_project/data/images/val/K-003544-004543-012247-016551_0_2_0_2_70_000_200.png: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     7.0293]



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:05<00:00,  3.80it/s]


                   all        295        890       0.71      0.953      0.852      0.827
            보령부스파정 5mg         24         24      0.668          1      0.916      0.909
           뮤테란캡슐 100mg          1          1      0.804          1      0.995      0.995
           일양하이트린정 2mg          2          2      0.447          1      0.497      0.497
    기넥신에프정(은행엽엑스)(수출용)         90         90      0.728          1      0.777      0.771
     무코스타정(레바미피드)(비매품)          5          5      0.627          1      0.862      0.829
                  알드린정         10         10      0.487          1      0.647      0.647
          뉴로메드정(옥시라세탐)         10         10      0.794          1      0.889      0.849
            타이레놀정500mg          4          4          1       0.45      0.699      0.643
          에어탈정(아세클로페낙)         11         11      0.885      0.818      0.931      0.872
         삼남건조수산화알루미늄겔정         35         35      0.796          1       0.87      0.859
타이레놀이알서방정(아세트아미노펜)(수출

## **6. 모델 평가**
- 학습된 YOLOv8x 모델의 최적 가중치(best.pt)를 활용하여, 테스트 이미지에 대한 객체 탐지 예측을 수행하였다.

- 예측에는 Ultralytics의 model.predict() 함수를 사용하였으며, confidence threshold를 0.05로 낮춰 검출 누락 가능성을 줄이는 방식으로 설정하였다.

In [ ]:
test_results = model.predict(
    source=test_dir,
    conf=0.05,
    imgsz=640,
    save=False,
    save_txt=False,
    name="predict_kaggle"
)


image 1/843 /content/drive/MyDrive/codeit_level1_project/data/test_images/1.png: 640x512 1 뉴로메드정(옥시라세탐), 1 삼남건조수산화알루미늄겔정, 1 리리카캡슐 150mg, 1 세비카정 10/40mg, 58.1ms
image 2/843 /content/drive/MyDrive/codeit_level1_project/data/test_images/10.png: 640x512 2 보령부스파정 5mgs, 1 타이레놀이알서방정(아세트아미노펜)(수출용), 1 조인스정 200mg, 1 리리카캡슐 150mg, 12.1ms
image 3/843 /content/drive/MyDrive/codeit_level1_project/data/test_images/100.png: 640x512 2 보령부스파정 5mgs, 1 삐콤씨에프정 618.6mg/병, 1 리리카캡슐 150mg, 1 엑스포지정 5/160mg, 13.1ms
image 4/843 /content/drive/MyDrive/codeit_level1_project/data/test_images/1003.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 비모보정 500/20mg, 1 로수바미브정 10/20mg, 12.1ms
image 5/843 /content/drive/MyDrive/codeit_level1_project/data/test_images/1004.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 비모보정 500/20mg, 1 로수바미브정 10/20mg, 12.4ms
image 6/843 /content/drive/MyDrive/codeit_level1_project/data/test_images/1005.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 비모보정 500/20mg, 1 로수바미브정 10/20mg, 12.2ms

## **7. 제출 파일 생성**

- 모델의 예측 결과를 기반으로 Kaggle 제출 포맷에 맞는 csv 파일을 생성한다.
- test_images 폴더의 이미지 파일명은 중간중간 누락된 번호가 많기 때문에, 단순한 순번(1, 2, 3...)이 아닌, 파일명에서 추출한 실제 숫자 ID를 그대로 사용해야 한다.
- 클래스 번호는 제출 시 반드시 주어진 category_id와 정확히 매핑해야 평가가 올바르게 이루어진다.

In [ ]:
# 클래스 매핑 로드
with open("/content/drive/MyDrive/codeit_level1_project/data/classid_to_categoryid.json", "r") as f:
    class_to_category = json.load(f)
    class_to_category = {int(k): v for k, v in class_to_category.items()}

# 결과 파싱 및 저장
rows = []
annotation_id = 1

for result in test_results:
    # image_id: 원본 이미지 경로에서 파일명 숫자 추출
    filename = result.path.split("/")[-1]
    image_id = int(filename.split(".")[0])  # 예: "0089.png" → 89

    boxes = result.boxes
    if boxes is None or boxes.xywh is None:
        continue

    for i in range(len(boxes.xywh)):
        x, y, w, h = boxes.xywh[i].tolist()
        score = float(boxes.conf[i])
        cls = int(boxes.cls[i])
        category_id = class_to_category.get(cls, -1)

        if category_id == -1:
            continue  # 매핑 실패한 클래스는 제외

        # COCO 포맷: x_min, y_min, width, height
        xmin = round(x - w / 2)
        ymin = round(y - h / 2)
        bbox_w = round(w)
        bbox_h = round(h)

        rows.append([
            annotation_id,
            image_id,
            category_id,
            xmin,
            ymin,
            bbox_w,
            bbox_h,
            round(score, 2)
        ])
        annotation_id += 1

# CSV 저장
df = pd.DataFrame(rows, columns=[
    "annotation_id", "image_id", "category_id",
    "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"
])
csv_path = "/content/submission.csv"
df.to_csv(csv_path, index=False)
print(f"CSV 저장 완료: {csv_path} / 총 {len(df)}개의 바운딩 박스")

CSV 저장 완료: /content/submission.csv / 총 3775개의 바운딩 박스
